## Sorting

### The `<=>` method

Any object in a collection that we want to sort **must** implement a `<=>` method

This method performs comparison between two objects of the same type and a returns a `-1`, `0` or `1` depending on whether the first is `less than`, `equal to` or `greater than` the second object; if the objects cannot be compared then `nil` is returned.  

The return value of the `<=>` is used by `sort` to determine the order in which to place the items.  if `<=>` returns `nil` to `sort` then it throws an argument error.

If you want to sort a collection that contains a particular type of object, you need to know two things;
1. Does the object type implement a `<=>` comparison method?
2. If yes, what is the specific implementation of that method for that object type (eg. `String#<=>` will be implemented differently to `Integer#<=>`)

**The ASCII Table**

The ASCII character order determines how we compared on ASCII character with another using the `String#<=>` method.

Some useful rules to remember are:

- Uppercase letters come before lowercase letters
- Digits and (most) punctuation come before letters
- There is an extended ASCII table containing accented and other characters - this comes after the main ASCII table

### The `sort` method

Using a block with the `sort` method gives us more control over how items are sorted.

The block needs two arguments passed to it (the two items to be compared) and the return value of the block has to be `-1`, `0`, `1` or `nil`.   

This code uses the `Integer#<=>` method in the block to perform the comparison.

In [3]:
[2, 5, 3, 4, 1].sort do |a, b|
  a <=> b
end

[1, 2, 3, 4, 5]

Switching the order in which the items are compared the new array that is returned is in descending order.  

In [4]:
[2, 5, 3, 4, 1].sort do |a, b|
  b <=> a
end

[5, 4, 3, 2, 1]

If the strings are of different lengths, and the strings are equal when compared up to the sortest length

**then the longer string is considered greater than the shorter one.**

In [5]:
['arc', 'bat', 'cape', 'ants', 'cap'].sort

["ants", "arc", "bat", "cap", "cape"]

Arrays are compared in an "element-wise" manner; the first element is `array_1` is compared with the first element of the other array using the `<=>` operator, then each of the second elements etc ...  As soon as the result of any such comparison is non-zero (i.e. the two corresponding elements are not equal), that result is returned for the whole array comparison.

If all the elements are equal, then the result is based on a comparison of the array lengths. 

```
Thus, two arrays are “equal” according to `Array#<=>` if, and only if, they have the same length and the value of each element is equal to the value of the corresponding element in the other array.
```

In [6]:
[['a', 'cat', 'b', 'c'], ['b', 2], ['a', 'car', 'd', 3], ['a', 'car', 'd']].sort

[["a", "car", "d"], ["a", "car", "d", 3], ["a", "cat", "b", "c"], ["b", 2]]

The sub-array with `'b'` as its first element, has a integer `2` as its second element.  Comparing an integer with a string will return `nil`, which will cause `sort` to throw an error.

In this case, `sort` did not need to compare the second element of that array in order to establish its order.  If the first element of that array was `a`, then the integer would come into play and `sort` would throw an error.

In [9]:
arrays = [['a', 'b', 'c'], ['a', 'b', 3]]
arrays.sort

ArgumentError: comparison of Array with Array failed

There is another sub-array which contains an integer `['a', 'car', 'd', 3]`. This integer does come into play, but only for comparing the length of `['a', 'car', 'd', 3]` to `['a', 'car', 'd']`.  The integer itself is not compared with a string, so no error is thrown.

### The `sort_by` method

`sort_by` is similar to `sort` but is usually called with a block.  The code in the block determines how the items are compared.

In [10]:
['cot', 'bed', 'mat'].sort_by do |word|
  word[1]
end

["mat", "bed", "cot"]

When calling `sort_by` on a hash, two arguments need to be passes to the block - the key and the value.

In [13]:
people = { Kate: 27, john: 25, Mike: 18 }

people.sort_by do |_, age|
  age
end

[[:Mike, 18], [:john, 25], [:Kate, 27]]

`sort_by` always return an array, even when called on a hash.  In the case of a hash it will return an array of nested arrays with key-value pairs as objects.

In order to order the hash by name, `sort` is using `Symbol#<=>` to peform the comparison between each name. `Symbol#<=>` Compares symbol with other_symbol after calling `to_s` on each of the symbols.

Since strings are compared in `ASCIIbetical` order, `'john` would come before `'Kate'` and `'Mike'` which may not be the desired result.  Use `String#Capitalize` on each key in the hash so when the keys are compared they are all capitalized.

In [14]:
people = { Kate: 27, john: 25, Mike:  18 }

people.sort_by do |name, _|
  name.capitalize
end

[[:john, 25], [:Kate, 27], [:Mike, 18]]

`Array#sort!` and `Array#sort_by!` are both destructive methods that return the same array but sorted.  These methods are specific to arrays and are not available to hashes.

## Nested Data Structures

### Referencing Collection Elements

To access the inner array element `3`, we need to reference it again, chaining our element references

In [18]:
arr = [[1, 3], [2]]
arr[0][1]


3

### Updating Collection Elements

The `arr[1] == 'hi there'` is a destructive method that permanently changed the second element in the `arr` array; it replace the array `[2]` with `'hi there'`

In [19]:
arr = [[1, 3], [2]]
arr[1] = "hi there"
arr

[[1, 3], "hi there"]

Modifying an element in a nested array looks similar to a chained reference, but it is not.  

The first part `arr[0]` is element reference and returns the array `[1,3]`.

The second part, `[1] == 5`, is the same as `[1, 3][1] == 5`.  This is a destructive action so the change is permanent.  

So, this is a chained action with two parts:
1. Element referenceand 
2. Element update

In [20]:
arr = [[1, 3], [2]]
arr[0][1] = 5
arr

[[1, 5], [2]]

In the code below, the line `arr[0] << 3` is also a two part chain:
1. The first part is element reference, which will return `[1]`
2. The second part is element update, and can be thought of as `[1] << 3`, which destructively appends `3` into the inner array.

In [21]:
arr = [[1], [2]]
arr[0] << 3
arr

[[1, 3], [2]]

### Other Nested Structures

Updating hash nested within an array follows the same two step process:
1. First we use `arr[0]` to reference the first hash in the array, which returns `{ a: 'ant' }`
2. Then we use the normal key/value hash creation syntax to create a new key/value pair: `{ a: 'ant' }[:c] == 'cat'`

This change is destructive and will be reflected in `arr`.

In [22]:
arr = [{ a: 'ant' }, { b: 'bear' }]
arr[0][:c] = 'cat'
arr

[{:a=>"ant", :c=>"cat"}, {:b=>"bear"}]

### Variable Reference for Nested Collections

The local variables `a` and `b` are pointing to `Array` objects.  

When we place the local variables as elements into the array, it looks the same as adding the actual array objects that they're pointing to into the array.  This is true to a certain extent.

In [28]:
a = [1, 3]
b = [2]
arr = [a, b]

arr

[[1, 3], [2]]

If we modify `a` after placing it in the array, this will also update the first element in the arr because `a` and `arr[0]` are still pointing to the same Array object.

In [29]:
a = [1, 3]
b = [2]
arr = [a, b]

a[1] = 5
arr 

[[1, 5], [2]]

If we modify the first array in `arr` this will also modify `a`.  In both cases, we are modifying the object that `a` and `arr[0]` are pointing to. We have two ways to reference the same object.

In [30]:
a = [1, 3]
b = [2]
arr = [a, b]

arr[0][1] = 8
puts arr.inspect 
puts a.inspect

[[1, 8], [2]]
[1, 8]


### Shallow Copy

Ruby provides two methods that allows us to copy an object, including collections: `dup` and `clone`. 

Both of these methods create a *shallow copy* of an object.  This means that 

 ```
 only the object that the method is called on is copied.  If the object contains other objects - like a nested array - then those object will be shared, not copied.
 ```

`dup` and `clone` allow objects within the copied object to be modified.

In [31]:
arr1 = ["a", "b", "c"]
arr2 = arr1.dup
arr2[1].upcase!

puts arr2.inspect
puts arr1.inspect

["a", "B", "c"]
["a", "B", "c"]


This happens because the destructive method (`String#upcase!`) was called on the object withib the array and not the array itself.  

Since those objects are `shared`, even if you mutate the object by referencing it from within a particular array or other collection, it is the `object` you are affecting rather than the collection.

In [ ]:
arr1 = ["a", "b", "c"]
arr2 = arr1.dup
arr2.map! do |char|
  char.upcase
end

puts arr2.inspect
puts arr1.inspect

`arr2` is changed by arr1 is not.  This is because the destructive method `Array#map!` is called on `arr2`.  This method modified the array, replacing each element of the array with a new value.

Since we changing the array (which is copied), and not the objects within the array (which are shared), `arr1` is left unchanged.

In [32]:
arr1 = ["a", "b", "c"]
arr2 = arr1.dup
arr2.each do |char|
  char.upcase!
end

puts arr2.inspect
puts arr1.inspect

["A", "B", "C"]
["A", "B", "C"]


The destructive `String#upcase!` is called on on each element of `arr2`.  However, every element in `arr2` is a reference to the object referenced by the corresponding element in `arr1`.  

### Freezing Objects

The main difference between `dup` and `clone` is that `clone` preserves the frozen state of the object.

In [33]:
arr1 = ["a", "b", "c"].freeze
arr2 = arr1.clone
arr2 << "d"

FrozenError: can't modify frozen Array: ["a", "b", "c"]

Only mutable objects can be frozen because immutable objects, like integers, are already frozen.  We can check if an object is frozen with the `frozen?` method.

`freeze` only freezes the object is called on.  If the object its called on contains other objects, those objects won't be frozen.   

In [34]:
arr = [[1], [2], [3]].freeze
arr[2] << 4
arr

[[1], [2], [3, 4]]

In [35]:
arr = [[1], [2], [3]].freeze
arr << [4]

FrozenError: can't modify frozen Array: [[1], [2], [3]]

## Working with Blocks

- Action:
- Object:
- Side Effect: 
- Return Value: 
- Is Return Value Used?:

### Example 1

In [36]:
[[1, 2], [3, 4]].each do |arr|
  puts arr.first
end

1
3


[[1, 2], [3, 4]]

Line 1:
- Action: method call `each`
- Object: Outer array
- Side Effect: None
- Return Value: The calling object `[[1, 2], [3, 4]]`
- Is Return Value Used?: No

Lines 1 - 3:
- Action: Block execution
- Object: Each sub-array
- Side Effect: None
- Return Value: `nil`
- Is Return Value Used?: No

Line 2:
- Action: method call `first`
- Object: Each sub-array
- Side Effect: None
- Return Value: Element at index 0 of sub-array
- Is Return Value Used?: Yes, used as argument for `puts`

Line 2:
- Action: method call `puts`
- Object: Return value of `arr.first`
- Side Effect: Outputs out string representation of integer
- Return Value: `nil`
- Is Return Value Used?: Yes, used to determine block's return value

### Example 2

In [37]:
[[1, 2], [3, 4]].map do |arr|
  puts arr.first
end

1
3


[nil, nil]

Line 1:
- Action: method call `map`
- Object: The outer array
- Side Effect: None
- Return Value: New array - `[nil, nil]`
- Is Return Value Used?: No

Lines 1 - 3:
- Action: Block execution
- Object: Each sub-array
- Side Effect: None
- Return Value: `nil`
- Is Return Value Used?: Yes, used by `map` for transformation

Line 2:
- Action: method call `first`
- Object: Each sub-array
- Side Effect: None
- Return Value: The element at index 0 of each sub-array
- Is Return Value Used?: Yes, passed as arguments to `puts`

Line 2:
- Action: method call `puts`
- Object: The element at index 0 of each sub-array
- Side Effect: Outputs string representation of integer
- Return Value: `nil`
- Is Return Value Used?: Yes, used to determine the return value of the block.

### Example 3

In [38]:
[[1, 2], [3, 4]].map do |arr|
  puts arr.first
  arr.first
end

1
3


[1, 3]

Line 1:
- Action: method call `map`
- Object: The outer array
- Side Effect: None
- Return Value: New array - `[1, 3]`
- Is Return Value Used?: No

Lines 1 - 4:
- Action: Block execution
- Object: Each sub-array
- Side Effect: None
- Return Value: The element at index 0 of each sub-array
- Is Return Value Used?: Yes, used by `map` for transformation

Line 2:
- Action: method call `first`
- Object: Each sub-array
- Side Effect: None
- Return Value: Element at index 0 of each sub-array
- Is Return Value Used?: Yes, passed as argument to `puts`

Line 2:
- Action: method call `puts`
- Object: Element at index 0 of each sub-array
- Side Effect: Outputs string representation of an integer
- Return Value: `nil`
- Is Return Value Used?: No

Line 3:
- Action: method call `first`
- Object: Each sub-array
- Side Effect: None
- Return Value: Element at index 0 of each sub-array
- Is Return Value Used?: Yes, used to determine the return value of block

### Example 4

In [ ]:
my_arr = [[18, 7], [3, 12]].each do |arr|
  arr.each do |num|
    if num > 5
      puts num
    end
  end
end

Line 1:
- Action: Variable assignment
- Object: The outer array `[[18, 7], [3, 12]]`
- Side Effect: None
- Return Value: Calling object `[[18, 7], [3, 12]]`
- Is Return Value Used?: No

Line 1:
- Action: method call `each`
- Object: The outer array `[[18, 7], [3, 12]]`
- Side Effect: None
- Return Value: Calling object `[[18, 7], [3, 12]]`
- Is Return Value Used?: Yes, used by variable assignment to `my_arr`

Line 1 - 7:
- Action: Outer block execution
- Object: Each sub-array
- Side Effect: None
- Return Value: Each sub-array
- Is Return Value Used?: No

Line 2:
- Action: method call `each`
- Object: Each sub-array
- Side Effect: None
- Return Value: Calling object - the sub-array in current iteration
- Is Return Value Used?: Yes, used to determine the return value of the outer block

Lines 2 - 6:
- Action: inner-block execution
- Object: Each interger in the sub-array of the current iteration
- Side Effect: None
- Return Value: `nil`
- Is Return Value Used?: No

Line 3: 
- Action: Comparison `num > 5`
- Object: Each interger in the sub-array of the current iteration
- Side Effect: None
- Return Value: Boolean
- Is Return Value Used?: Yes, evaluated to `if`

Line 3: 
- Action: Conditional `if`
- Object: The result of `num > 5` (boolean)
- Side Effect: None
- Return Value: `nil`
- Is Return Value Used?: Yes, used to determine the return value of the inner block

Line 4:
- Action: method call `puts`
- Object: Each interger in the sub-array of the current iteration
- Side Effect: Outputs string representation of an integer
- Return Value: `nil`
- Is Return Value Used?: Yes, used to determine the return value of the conditional statement if the condition is met.



### Example 5

In [ ]:
[[1, 2], [3, 4]].map do |arr|
  arr.map do |num|
    num * 2
  end
end

Line 1:
- Action: method call `map`
- Object: Outer array `[[1, 2], [3, 4]]`
- Side Effect: None
- Return Value: New transformed array - `[[2, 4], [6, 8]]`
- Is Return Value Used?: No

Lines 1 - 5:
- Action: Outer block execution
- Object: Each sub-array
- Side Effect: None
- Return Value: The transformed sub-arrays
- Is Return Value Used?: Yes, by top-level `map` for transformation

Line 2:
- Action: method call `map`
- Object: Each sub-array
- Side Effect: None
- Return Value: New transformed sub-array of that iteration
- Is Return Value Used?: Yes, used to determine the return value of the outer block

Lines 2 - 4:
- Action: Inner block execution
- Object: The element in the sub-array on that iteration
- Side Effect: None
- Return Value: Integer
- Is Return Value Used?: Yes, it is used by inner `map` for transformation

Line 3:
- Action: method call `Integer#*` with integer 2 as an argument
- Object: The element in the sub-array on that iteration
- Side Effect: None
- Return Value: Integer
- Is Return Value Used?: Yes, it is used to determine the return value of the inner block.


### Example 6

We want to select all elements where every key matches the first letter of the value. Note that the keys are symbols here, so we'll have to do some conversion before comparison.

In [87]:
arr = [{ a: 'ant', b: 'elephant' }, { c: 'cat' }]

arr.select do |hsh|
  hsh.all? do |k, v|
    k.to_s == v[0]
  end
end

[{:c=>"cat"}]

Line 3:
- Action: method call `select`
- Object: The outer array 
- Side Effect: None
- Return Value: A new array
- Is Return Value Used?: No

Lines 3 - 7:
- Action: Outer block exection
- Object: Each hash
- Side Effect: None
- Return Value: Boolean
- Is Return Value Used?: Yes, used by `select` to determine selection criteria

Line 4:
- Action: method call `all?`
- Object: Each hash
- Side Effect: None
- Return Value: Boolean
- Is Return Value Used?: Yes, used to determine the return value of the outer block

Lines 4 - 6:
- Action: Inner block execution
- Object: The key/value pairs in the hash of the current iteration
- Side Effect: None
- Return Value: Boolean
- Is Return Value Used?: Yes, used by `all?` to determine truthiness

Line 5:
- Action: method call `to_s`
- Object: The current key
- Side Effect: None
- Return Value: A string
- Is Return Value Used?: Yes, used in comparison

Line 5:
- Action: Element reference `v[0]`
- Object: The current value
- Side Effect: None
- Return Value: The string at index 0 of current value
- Is Return Value Used?: Yes, used in comparison

Line 5:
- Action: Comparison `k.to_s == v[0]`
- Object: The current key (current value is passed as argument)
- Side Effect: None
- Return Value: Boolean
- Is Return Value Used?: Yes, used to determine the return value of inner block.

Store each key/value pairs where the `key.to_s == value[0]` in a separate dictionary and store it in a list

In [89]:
arr = [{ a: 'ant', b: 'elephant', d: 'donkey' }, { c: 'cat' }]

arr.map do |hsh|
  hsh.select do |k, v|
    k.to_s == v[0]
  end
end

[{:a=>"ant", :d=>"donkey"}, {:c=>"cat"}]

In [94]:
arr = [{ a: 'ant', b: 'elephant', d: 'donkey' }, { c: 'cat' }]
matches = []

arr.each do |hsh|
  hsh.each_with_object({}) do |(k, v), sub_hsh|
    if k.to_s == v[0]
      matches << {k => v}
    end
  end
end

[{:a=>"ant", :b=>"elephant", :d=>"donkey"}, {:c=>"cat"}]

### Example 7

We have an array of nested arrays which contain numeric strings, and we want to sort the outer array so that the inner arrays are ordered according to the numeric value of the strings they contain. Take, for example, the following 4-element array of arrays.

In [84]:
arr = [['1', '8', '11'], ['2', '6', '13'], ['2', '12', '15'], ['1', '8', '9']]

arr.sort_by do |sub_arr|
  sub_arr.map(&:to_i)
end

[["2", "12", "15"], ["2", "6", "13"], ["1", "8", "11"], ["1", "8", "9"]]

In [83]:
arr = [['1', '8', '11'], ['2', '6', '13'], ['2', '12', '15'], ['1', '8', '9']]

arr.sort_by do |sub_arr|
  sub_arr.map do |str_num|
    -str_num.to_i
  end
end

[["2", "12", "15"], ["2", "6", "13"], ["1", "8", "11"], ["1", "8", "9"]]

### Example 8

Take the 2-element array below, where we only want to select integers greater than 13 but strings less than 6 characters.

In [97]:
arr = [[8, 13, 27], ['apple', 'banana', 'cantaloupe']]

arr.map do |sub_arr|
  sub_arr.select do |elem|
    if elem.is_a?(Integer)
      elem > 13
    else
      elem.size < 6
    end
  end
end

[[27], ["apple"]]

In [98]:
arr.map.with_index do |sub_arr, idx|
  sub_arr.select do |elem|
    if idx == 0
      elem > 13
    else
      elem.size < 6
    end
  end
end

[[27], ["apple"]]

### Example 10

Increment every number by 1 without changing the data structure.

In [103]:
arr = [
  [[1, 2], [3, 4]], 
  [5, 6]
]

[[[1, 2], [3, 4]], [5, 6]]

In [104]:
new_arr = arr.map do |sub_arr|
  sub_arr.map do |elem|
    if elem.is_a?(Integer)
      elem + 1
    else
      elem.map do |sub_elem|
        sub_elem + 1
      end
    end
  end
end

puts arr.inspect
puts new_arr.inspect

[[[1, 2], [3, 4]], [5, 6]]
[[[2, 3], [4, 5]], [6, 7]]


### Mutating Collections While Iterating 

**Do not mutate the collection that you're iterating through**

Here's an example of what not to do:

In [106]:
def remove_evens!(arr)
  arr.each do |num|
    if num % 2 == 0
      arr.delete(num)
    end
  end
  arr
end

puts remove_evens!([1,1,2,3,4,6,8,9]).inspect

# expected return value [1, 1, 3, 9]

[1, 1, 3, 6, 9]


## Practice Problems

### Practice Problem 1

How would you order this array of number strings by descending numeric value?

In [110]:
arr = ['10', '11', '9', '7', '8']

arr.sort_by do  |string_num| 
  -string_num.to_i
end

["11", "10", "9", "8", "7"]

Sorting is carried out by comparing the items in a collection with each other, and ordering them based on the result of that comparison.  

The `<=>` method performs comparison between two objects of the same type and returns a `-1` if the first value is less than the second, `0` if they are even, `1` if the first item is greater than the second or `nil` if the items cannot be compared.  The return value of `<=>` is used by `sort` to determine the order in which to place the items.

The `sort_by` method is called on the array.  The code in the block determines how the items are compared.  In this case, we want to sort the strings within the array in descending order according to their numeric values. The ASCII character order determines the result when we compare one string with another using the `String#<=>` method ('ASCIIbetical' order).  Because of this, we must convert the strings to integers before invoking the `<=>` method.  

In addtion to converting the strings to integers, we also change the sign of the integer by prepending the `'-'` symbol to the return value of `string_num.to_i`.  This negates the value of each integer and reverses the order in which the items are sorted.


### Practice Problem 2

How would you order this array of hashes based on the year of publication of each book, from the earliest to the latest?

In [113]:
books = [
  {title: 'One Hundred Years of Solitude', author: 'Gabriel Garcia Marquez', published: '1967'},
  {title: 'The Great Gatsby', author: 'F. Scott Fitzgerald', published: '1925'},
  {title: 'War and Peace', author: 'Leo Tolstoy', published: '1869'},
  {title: 'Ulysses', author: 'James Joyce', published: '1922'}
]

books.sort_by do |hsh|
  hsh[:published]
end

[{:title=>"War and Peace", :author=>"Leo Tolstoy", :published=>"1869"}, {:title=>"Ulysses", :author=>"James Joyce", :published=>"1922"}, {:title=>"The Great Gatsby", :author=>"F. Scott Fitzgerald", :published=>"1925"}, {:title=>"One Hundred Years of Solitude", :author=>"Gabriel Garcia Marquez", :published=>"1967"}]

Sorting is carried out by comparing the items of a collection with eachother, and ordering them based on the result of that comparison.

The `<=>` performs the comparison between two objects of the same type and returns:
- `-1` if the first item is less than the second
- `0` if the first item is equal to the second
- `1` if the first item is greater than the second item
- `nil` if the onbjects can't be compared.

The `sort_by` method is called on the array.  The code in the block determines how the items are compared.  Each hash is passed to the block as an argument and the block returns the value associated with the `:published` key in each hash.  

The string values do not need to be converted to integers because they are all four characters in length.  Since strings are compared character-by-character, we can just compared characters of each string at similar index positions.

### Practice Problem 3

For each of these collection objects demonstrate how you would reference the letter `'g'`.

In [117]:
arr1 = ['a', 'b', ['c', ['d', 'e', 'f', 'g']]]

arr1[2][1][-1]  # Chaining element references

"g"

In [128]:
arr2 = [{first: ['a', 'b', 'c'], second: ['d', 'e', 'f']}, {third: ['g', 'h', 'i']}]

arr2[1][:third][0]

"g"

In [130]:
arr3 = [['abc'], ['def'], {third: ['ghi']}]

arr3[2][:third][0][0]

"g"

In [132]:
hsh1 = {'a' => ['d', 'e'], 'b' => ['f', 'g'], 'c' => ['h', 'i']}

hsh1['b'][1]

"g"

In [146]:
hsh2 = {first: {'d' => 3}, second: {'e' => 2, 'f' => 1}, third: {'g' => 0}}

hsh2[:third].key(0)

"g"

### Practice Problem 4

For each of these collection objects where the value `3` occurs, demonstrate how you would change this to `4`.

In [160]:
arr1 = [1, [2, 3], 4]

arr1[1][1] = 4

arr1

[1, [2, 4], 4]

In [159]:
arr2 = [{a: 1}, {b: 2, c: [7, 6, 5], d: 4}, 3]

arr2[2] = 4

arr2

[{:a=>1}, {:b=>2, :c=>[7, 6, 5], :d=>4}, 4]

In [158]:
hsh1 = {first: [1, 2, [3]]}

hsh1[:first][2][0] = 4

hsh1

{:first=>[1, 2, [4]]}

In [157]:
hsh2 = {['a'] => {a: ['1', :two, 3], b: 4}, 'b' => 5}

hsh2[['a']][:a][2] = 4

hsh2

{["a"]=>{:a=>["1", :two, 4], :b=>4}, "b"=>5}

### Practice Problem 5

Given this nested Hash:

figure out the total age of just the male members of the family.

In [174]:
munsters = {
  "Herman" => { "age" => 32, "gender" => "male" },
  "Lily" => { "age" => 30, "gender" => "female" },
  "Grandpa" => { "age" => 402, "gender" => "male" },
  "Eddie" => { "age" => 10, "gender" => "male" },
  "Marilyn" => { "age" => 23, "gender" => "female"}
}

men = munsters.each_value.select do |v|
  v['gender'] == 'male'
end

men.sum { |hsh| hsh['age'] }

444

In [167]:
munsters = {
  "Herman" => { "age" => 32, "gender" => "male" },
  "Lily" => { "age" => 30, "gender" => "female" },
  "Grandpa" => { "age" => 402, "gender" => "male" },
  "Eddie" => { "age" => 10, "gender" => "male" },
  "Marilyn" => { "age" => 23, "gender" => "female"}
}

total = 0

munsters.each_value do |v|
  total += v['age'] if v['gender'] == 'male'
end

total

444

In [172]:
male_ages = munsters.each_with_object([]) do |(_, v), arr|
  arr << v['age'] if v['gender'] == 'male'
end

male_ages.sum

444

### Practice Problem 6

Given this previously seen family hash, print out the name, age and gender of each family member like this:

```
(Name) is a (age)-year-old (male or female).
```

In [177]:
munsters = {
  "Herman" => { "age" => 32, "gender" => "male" },
  "Lily" => { "age" => 30, "gender" => "female" },
  "Grandpa" => { "age" => 402, "gender" => "male" },
  "Eddie" => { "age" => 10, "gender" => "male" },
  "Marilyn" => { "age" => 23, "gender" => "female"}
}

munsters.each do |k, v|
  age, gender = v.values
  puts "#{k} is a #{age}-year-old #{gender}."
end

Herman is a 32-year-old male.
Lily is a 30-year-old female.
Grandpa is a 402-year-old male.
Eddie is a 10-year-old male.
Marilyn is a 23-year-old female.


{"Herman"=>{"age"=>32, "gender"=>"male"}, "Lily"=>{"age"=>30, "gender"=>"female"}, "Grandpa"=>{"age"=>402, "gender"=>"male"}, "Eddie"=>{"age"=>10, "gender"=>"male"}, "Marilyn"=>{"age"=>23, "gender"=>"female"}}

### Practice Problem 7

Given this code, what would be the final values of a and b? Try to work this out without running the code.

In [178]:
a = 2
b = [5, 8]
arr = [a, b] 

arr[0] += 2 
arr[1][0] -= a

arr

[4, [3, 8]]

### Practice Problem 8

Using the each method, write some code to output all of the vowels from the strings.

In [180]:
hsh = {first: ['the', 'quick'], second: ['brown', 'fox'], third: ['jumped'], fourth: ['over', 'the', 'lazy', 'dog']}

hsh.each do |_, v|
  v.each do |string|
    string.chars.each do |char|
      puts char if 'aeiou'.include?(char)
    end
  end
end


e
u
i
o
o
u
e
o
e
e
a
o


{:first=>["the", "quick"], :second=>["brown", "fox"], :third=>["jumped"], :fourth=>["over", "the", "lazy", "dog"]}

In [ ]:
hsh = {first: ['the', 'quick'], second: ['brown', 'fox'], third: ['jumped'], fourth: ['over', 'the', 'lazy', 'dog']}

hsh.each_value do |arr|
  arr.each do |string|
    puts string.chars.select { |char| 'aeiou'.include?(char) }
  end
end

### Practice Problem 9

Given this data structure, return a new array of the same structure but with the sub arrays being ordered (alphabetically or numerically as appropriate) in descending order.

In [192]:
arr = [['b', 'c', 'a'], [2, 1, 3], ['blue', 'black', 'green']]

arr.map do |sub_arr|
  sub_arr.sort_by do |char|
    -char.ord
  end
end

[["c", "b", "a"], [3, 2, 1], ["green", "blue", "black"]]

In [190]:
arr = [['b', 'c', 'a'], [2, 1, 3], ['blue', 'black', 'green']]

arr.map do |sub_arr|
  if sub_arr.all?{ |char| char.is_a?(Integer) }
    sub_arr.sort_by { |char| -char }
  else
    sub_arr.sort_by { |char| -char.ord }d
  end
end

[["c", "b", "a"], [3, 2, 1], ["green", "blue", "black"]]

In [193]:
arr = [['b', 'c', 'a'], [2, 1, 3], ['blue', 'black', 'green']]

arr.map do |sub_arr|
  sub_arr.sort do |a, b|
    b <=> a
  end
end

[["c", "b", "a"], [3, 2, 1], ["green", "blue", "black"]]

### Practice Problem 10

Given the following data structure and without modifying the original array, use the map method to return a new array identical in structure to the original but where the value of each integer is incremented by 1.

In [198]:
arr = [{a: 1}, {b: 2, c: 3}, {d: 4, e: 5, f: 6}]

arr.map do |hash|
  hash.each_with_object({}) do |(k, v), hsh|
    hsh[k] = v + 1
  end
end

[{:a=>2}, {:b=>3, :c=>4}, {:d=>5, :e=>6, :f=>7}]

In [201]:
arr = [{a: 1}, {b: 2, c: 3}, {d: 4, e: 5, f: 6}]

arr.map do |hash|
  temp_hash = {}
  hash.each do |k, v|
    temp_hash[k] = v + 1
  end
  temp_hash
end

[{:a=>2}, {:b=>3, :c=>4}, {:d=>5, :e=>6, :f=>7}]

### Practice Problem 11

Given the following data structure use a combination of methods, including either the select or reject method, to return a new array identical in structure to the original but containing only the integers that are multiples of 3.

In [202]:
arr = [[2], [3, 5, 7], [9], [11, 13, 15]]

arr.map do |sub_arr|
  sub_arr.select do |num|
    num % 3 == 0
  end
end

[[], [3], [9], [15]]

In [203]:
arr = [[2], [3, 5, 7], [9], [11, 13, 15]]

arr.map do |sub_arr|
  sub_arr.reject do |num|
    num % 3 != 0 
  end
end

[[], [3], [9], [15]]

### Practice Problem 12

Given the following data structure, and without using the Array#to_h method, write some code that will return a hash where the key is the first item in each sub array and the value is the second item.

In [ ]:
arr = [[:a, 1], ['b', 'two'], ['sea', {c: 3}], [{a: 1, b: 2, c: 3, d: 4}, 'D']]

arr.each_with_object({}) do |(one, two), hsh|
  hsh[one] = two
end

In [211]:
arr = [[:a, 1], ['b', 'two'], ['sea', {c: 3}], [{a: 1, b: 2, c: 3, d: 4}, 'D']]

arr.each_with_object({}) do |sub_arr, hsh|
  hsh[sub_arr[0]] = sub_arr[1]
end

{:a=>1, "b"=>"two", "sea"=>{:c=>3}, {:a=>1, :b=>2, :c=>3, :d=>4}=>"D"}

In [214]:
arr = [[:a, 1], ['b', 'two'], ['sea', {c: 3}], [{a: 1, b: 2, c: 3, d: 4}, 'D']]
new_hash = {}

arr.each do |sub_arr|
  new_hash[sub_arr[0]] = sub_arr[1]
end

new_hash

{:a=>1, "b"=>"two", "sea"=>{:c=>3}, {:a=>1, :b=>2, :c=>3, :d=>4}=>"D"}

### Practice Problem 13

Given the following data structure, return a new array containing the same sub-arrays as the original but ordered logically by only taking into consideration the odd numbers they contain.

```ruby 
arr = [[1, 6, 9], [6, 1, 7], [1, 8, 3], [1, 5, 9]]

[[1, 8, 3], [1, 5, 9], [6, 1, 7], [1, 6, 9]]
```

In [217]:
arr = [[1, 6, 9], [6, 1, 7], [1, 8, 3], [1, 5, 9]]

arr.sort_by do |sub_arr|
  sub_arr.select do |num|
    num.odd?
  end
end

[[1, 8, 3], [1, 5, 9], [6, 1, 7], [1, 6, 9]]

In [218]:
arr = [[1, 6, 9], [6, 1, 7], [1, 8, 3], [1, 5, 9]]

arr.sort_by { |sub_arr| sub_arr.select(&:odd?) }

[[1, 8, 3], [1, 5, 9], [6, 1, 7], [1, 6, 9]]

### Practice Problem 14

Given this data structure write some code to return an array containing the colors of the fruits and the sizes of the vegetables. 

The sizes should be uppercase and the colors should be capitalized.

```Ruby
[["Red", "Green"], "MEDIUM", ["Red", "Green"], ["Orange"], "LARGE"]
```

In [219]:
hsh = {
  'grape' => {type: 'fruit', colors: ['red', 'green'], size: 'small'},
  'carrot' => {type: 'vegetable', colors: ['orange'], size: 'medium'},
  'apple' => {type: 'fruit', colors: ['red', 'green'], size: 'medium'},
  'apricot' => {type: 'fruit', colors: ['orange'], size: 'medium'},
  'marrow' => {type: 'vegetable', colors: ['green'], size: 'large'},
}

hsh.map do |k, v|
  if v[:type] == 'fruit'
    v[:colors].map(&:capitalize)
  else
    v[:size].upcase
  end
end

[["Red", "Green"], "MEDIUM", ["Red", "Green"], ["Orange"], "LARGE"]

### Practice Problem 15

Given this data structure write some code to return an array which contains only the hashes where all the integers are even.

In [220]:
arr = [{a: [1, 2, 3]}, {b: [2, 4, 6], c: [3, 6], d: [4]}, {e: [8], f: [6, 10]}]

arr.select do |hsh|
  hsh.all? do |_, v|
    v.all? do |num|
      num.even?
    end
  end
end

[{:e=>[8], :f=>[6, 10]}]

In [221]:
arr = [{a: [1, 2, 3]}, {b: [2, 4, 6], c: [3, 6], d: [4]}, {e: [8], f: [6, 10]}]

arr.select do |hsh|
  hsh.all? do |_, v|
    v.all?(&:even?)
  end
end

[{:e=>[8], :f=>[6, 10]}]